In [16]:
import pandas as pd
from pandas import DataFrame
from pandas import Series

# Import dataset
services = pd.read_csv('Aurora_Office_Fase_2_-_all_versions_-_False_-_2023-09-20-21-01-20.csv', sep=';', index_col = False)

# rename variables
newColumns = {'_coordenadas_latitude': 'lat', 
              '_coordenadas_longitude' : 'long',  
              'pais' : 'ubicacion' , 
              'punto_reporte' : 'lugar_enc', 
              'ubicacion' : 'punto_ayud', 
              'cualotropunto' : 'punto_ayud_otro', 
              'infraestructura' : 'cuenta_con' , 
              'sexo' : 'identifica', 
              'organizacionpertence' : 'org_pert', 
              'infraestructuradiscapacidad' : 'serv_disca' , 
              'servicios' : 'serv_tipo',
              'siserviciosnna' : 'serv_dif_n',
              'serviciosnna' : 'cual_serv1',
              'otroservicionna' : 'otro_cual_',
              'siserviciosmujeres' : 'serv_dif_m',
              'serviciosmujeres' : 'cual_ser_2',
              'otroserviciomujeres' : 'otro_dif_m',
              'retos' : 'princ_reto',
              'idiomas' : 'idioma_ent',
               'medios' :  'medios_bri',
               'diarioasesorialegal' : 'cap_serv_l',
               'ayerasesorialegal' : 'cap_serv_1',
              'diarioalimentacionynutricion' : 'tipo_alime',
              'ayeralimentacionynutricion' : 'pers_alim',
              'diariosaludprimerosauxiliosyatencionmedica' : 'pers_atien',
              'ayersaludprimerosauxiliosyatencionmedica' : 'serv_sal_1',
              'diarioaguapotable' : 'prom_agua_',
              'ayeraguapotable' : 'serv_agua',
              'diariosaneamiento' : 'serv_san_d',
              'ayersaneamiento' : 'serv_san_a',
              'diariohigiene' : 'serv_hig_d',
              'ayerhigiene' : 'serv_hig_a',
              'diarioeducacion' : 'serv_edu_d',
              'ayereducacion' : 'serv_edu_a',
              'diarioalojamientotemporal' : 'serv_aloj',
              'ayeralojamientotemporal' : 'serv_aloj_',
              'diarioapoyooayudapsicosocial' : 'serv_psico',
              'ayerapoyooayudapsicosocial' : 'serv_psi_1',
              'diariotransportehumanitario' : 'serv_trans',
              'ayertransportehumanitario' : 'serv_tra_1',
              'diarioserviciosproteccion' : 'serv_prot_c', #new variable before was taken from "other" services
              'ayerserviciosproteccion' : 'serv_prot_a', #new variable before was taken from "other"  services
              'diariorestablecimientodecontactofamiliar' : 'serv_esta_c', #new variable before was taken from "other" services
              'ayerrestablecimientodecontactofamiliar' : 'serv_esta_a',  #new variable before was taken from "other" services
              'diariotransferenciasmonetarias' : 'serv_trans_c',
              'ayertransferenciasmonetarias' : 'serv_trans_a',
              'diariootro' : 'serv_tra_2',
              'ayerotro' : 'atend_serv',

              }
services_carto = services.rename(columns=newColumns)

C:\Users\maola\AppData\Local\Temp\ipykernel_16780\1023239090.py:6: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  services = pd.read_csv('Aurora_Office_Fase_2_-_all_versions_-_False_-_2023-09-20-21-01-20.csv', sep=';', index_col = False)


In [17]:
def codifyServices(value: str, values_dict: dict[str, int], otherValue:str):
    if(type(value) == float):
        return ""
    raw_values = value.split(" ")
    output = []
    for value in raw_values:
        try:      
            codedValue = values_dict[value]
            output.append(str(codedValue) )
        except Exception as e:
            output.append(otherValue)
            
    return "|".join(output)

def processColumn(dfColumn:Series, values_dict: dict[int, str], other_value: str):
    reversed_values_dict = dict( [( x[1],x[0])  for x in values_dict.items()])
    return dfColumn.apply(lambda x: codifyServices(x, reversed_values_dict, other_value))

def processMultValueColumns(df: DataFrame, columnObjectsList: list[dict]):
    """
    df: DataFrame object
    columnsObjectsList: list of column object
    columnObject: dictionary {"target_column": str, "output_column": str, values_dict: dict, other_value: str}

    return DataFrame Object
    """
    for columnObject in columnObjectsList:
        try:
            target_column = columnObject["target_column"]
            output_column = columnObject["output_column"]
            values_dict = columnObject["values_dict"]
            other_value = columnObject["other_value"]
            df[output_column] =  processColumn(df[target_column], values_dict, other_value)
        except Exception as e:
            print(e)
            continue
    return df

def exportToFile(df: DataFrame,fileType: str, exportName: str):
    """ 
    df -> Pandas DataFrame object
    fileType -> Either "csv" or "json"
    exportName -> File location
    """
    if(fileType == "csv"):
        name = f"{exportName}.csv"
        df.to_csv(name)
        print (f"data export to {name}")
    else:
        name = f"{exportName}.json"
        df.to_json(name, orient="records")
        print (f"data export to {name}")
    

In [14]:
# serv_tipo (separating by pipe symbol and categorized with number)

services_dict = {
  1 :"asesorialegal",
  2 :"alimentacionynutricion",
  3 :"saludprimerosauxiliosyatencionmedica",
  4 :"aguapotable",
  5 :"saneamiento",
  6 :"higiene",
  8 :"educacion",
  9 :"alojamientotemporal",
  11 :"acompañamientoayudapsicosocial",
  12 :"transportehumanitario",
  13 :"Otro",
  14 : "serviciosdeproteccion",
  16 : "restablecimientodecontactofamiliar",
  18 : "transferencias",  # new service: "Transferencias Monetarias"
      
}

# re structure variable cuenta_con
cuenta_con_dict = {
  1 :"accesoaserviciodeinternet",
  2 :"accesoaservicioelectrico",
  3 :"areadecomedorescocina",
  4 :"areaparadisposiciondebasuras",
  5 :"areascomunes",
  6 :"areasdelavanderia",
  7 :"areasderegistrodelapoblacion",
  8 :"ningunadelasanteriores",
}

# re structure variable children services (cual_serv1)
cual_serv1_dict = {
  1 :"nnaapoyooayudapsicosocial",
  2 :"nnagestiondecasos",
  3 :"nnaentregadekits",
  4 :"nnaalimentacionynutricion",
  5 :"nnabanosexclusivosnna", 
  6 :"nnacuidadosalternativos",
  7 :"nnaotro",
  8 :"espaciosamigables", # New option: Espacios Amigables
  9 : " ",
}

# re structure variable women services (cual_ser_2)
cual_ser_2_dict = {
  0 : " ",
  1 :"mujeresespaciosseguros",
  2 :"mujeresduchasobanos",
  3 :"mujeresgestiondecasosdeviolenciasbasadasengenero",
  4 :"mujeresproductosdehigienemenstrual",
  5 :"mujeresotro",
  6 :"mujerescontrolesprenatales", # New option: Controles prenatales para mujeres gestantes 
  7 :"mujeresapoyonutricionalgestantesylactantes", #New option: Apoyo nutricional para mujeres lactantes y/o gestantes

}

# re structure variable data storage (almacenamientoregistros)
registro_dict = {
  1 :"archivofisico",
  2 :"hojacalculo",
  3 :"software",
  4 :"otroalmacenarregistro",
  5 : " ",
}

# variable funding 
financ_dict = {
  1 :"organismosorganizacionesinternacionales",
  2 :"gobiernonacional",   
  3 :"empresasnacionales",
  4 :"empresasinternacionales",
  5 :"organismosnogubernamentaleslocalesonacionales",
  6 :"sociedadcivil",
  7 :"otrosfinanciamientos",

}

#  variable challenges
reto_dict = {
1 :"conseguirlafinanciacionnecesaria",
2 :"mejorarlacalidaddelosserviciosofrecidos",
3 :"ampliarymejorarlacolaboracionconotrasentidades",
4 :"mejorarlavisibilidaddeltrabajorealizado",
5 :"medirycomunicarelimpactosocialdelosserviciosprestados",
6 :"comunicacionconlosusuarios",
7 :"otroreto",
}

# variable lenguages
idio_dict = {
1 :"espanol",
2 :"ingles",
3 :"creole",
4 :"portugues",
5 :"otroidioma",
6 :"frances", 
}

# variable medios
medio_dict = {
1 :"caraacara",
2 :"folletos",
3 :"infografiaeninstalacionesdelpunto",
4 :"mediosaudiovisuales",
5 :"redessocialeseinternet",
6 :"otromedio"
}

values = [
    {
        "target_column": "serv_tipo",
        "output_column": "serv_tipo1",
        "values_dict": services_dict,
        "other_value": "0"
    },
    {
        "target_column": "cuenta_con",
        "output_column": "cuenta_c_1",
        "values_dict": cuenta_con_dict,
        "other_value": "0"
    },
    {
        "target_column": "cual_serv1",
        "output_column": "cual_ser_1",
        "values_dict": cual_serv1_dict,
        "other_value": "0"
    },
    {
        "target_column": "cual_ser_2",
        "output_column": "cual_ser_3",
        "values_dict": cual_ser_2_dict,
        "other_value": "0"
    },
    {
        "target_column": "almacenamientoregistros",
        "output_column": "almacenamientoregistros_",
        "values_dict": cual_ser_2_dict,
        "other_value": "0"
    },
     {
        "target_column": "financiamiento",
        "output_column": "financb",
        "values_dict": financ_dict,
        "other_value": "0"
    },
         {
        "target_column": "princ_reto",
        "output_column": "princ_re_1",
        "values_dict": reto_dict ,
        "other_value": "0"
    },
             {
        "target_column": "idioma_ent",
        "output_column": "idioma_e_1",
        "values_dict": idio_dict ,
        "other_value": "0"
    },

                 {
        "target_column": "medios_bri",
        "output_column": "medios_b_1",
        "values_dict": medio_dict ,
        "other_value": "0"
    },
]




In [18]:
output_df = processMultValueColumns(services_carto, values)
exportToFile(output_df, "csv", "output")

data export to output.csv
